In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, Literal
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import operator

load_dotenv()

model= ChatOpenAI(model='gpt-4o-mini')

In [ ]:
class SentimentSchema(BaseModel):
    sentiment: Literal['positive', 'negative']=Field(description="Sentiment of the review")


In [ ]:
structured_model=model.with_structured_output(SentimentSchema)

In [ ]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal['UX','Performance','Bug','Support','Other'] = Field(description='The category of issue mentioned in the review')
    tone: Literal['angry','frustrated','disappointed','calm'] = Field(description='The emotional tone expressed by the user')
    urgency: Literal['low','medium','high'] = Field(description='How urgent or critical the issue appears to be')

In [ ]:
structured_model2=model.with_structured_output(DiagnosisSchema)

In [ ]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal['positive', 'negative']
    diagnosis: dict
    response: str

In [ ]:
def find_sentiment(state: ReviewState):

    prompt=f'For the following review find out the sentiment\n {state["review"]}'
    sentiment=structured_model.invoke(prompt).sentiment
    return {'sentiment':sentiment}

In [ ]:
def check_sentiment(state: ReviewState) -> Literal['positive_response','run_diagnosis']:
    if state['sentiment'] == 'positive':
        return 'positive_response'
    else:
        return 'run_diagnosis'


In [ ]:
def positive_response(state: ReviewState):
    prompt= f'''Write a warm thank you message in response to this review: \n\n'{state["review"]}\'\n
    Also kindly ask the user to leave feedback on our website'''

    response=model.invoke(prompt).content

    return {'response':response}

In [ ]:
def run_diagnosis(state: ReviewState):
    prompt=f'''Diagnose this negative review:\n\n{state['review']}\n'
            'Return issue_type, tone, urgency'''
    response=structured_model2.invoke(prompt)

    return {'diagnosis':response.model_dump()}


In [ ]:
def negative_response (state: ReviewState):
    diagnosis= state['diagnosis']
    prompt=f'''You are a support assistant.
            The user has a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'. Write an empathetic, helpful resolution message.'''
    response=model.invoke(prompt).content

    return {'response':response}

In [ ]:
#create the graph
graph= StateGraph(ReviewState)

#add nodes
graph.add_node('find_sentiment',find_sentiment)
graph.add_node('positive_response',positive_response)
graph.add_node('run_diagnosis',run_diagnosis)
graph.add_node('negative_response',negative_response)

#add edge
graph.add_cond(START, 'find_sentiment')

#conditional edges
graph.add_conditional_edges('find_sentiment', check_sentiment)

#add edge
graph.add_edge('positive_sentiment', END)
graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response',END)

#compile
workflow=graph.compile()

In [ ]:
#to see the workflow
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png())

In [ ]:
#execute (for positive sentiment check)

initial_state={'review': "I have been using this app for about a month now, and I must say, the user interface is incredible. Great job to the design team" }

final_state=workflow.invoke(initial_state)

print(final_state)

In [ ]:
#execute (for negative sentiment check)

initial_state={'review': "I have been trying to log in for an hour now, the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality." }

final_state=workflow.invoke(initial_state)

print(final_state)